In [25]:
import pandas as pd
pd.options.display.float_format = '{:.3f}'.format
pd.set_option("display.max_rows", None)  # 모든 행 출력
pd.set_option("display.max_columns", None)  # 모든 열 출력
import numpy as np
import matplotlib.pyplot as plt
import gurobipy as gp
from gurobipy import GRB
import os
from itertools import product
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from tqdm import tqdm
from functions import (load_parameters, load_generation_data, load_price_data, generate_randomized_generation,
generate_rt_scenarios, plot_generation_data, plot_randomized_generation, plot_scenarios_for_generator, plot_rt_scenarios, plot_summary)

generation_data, I, T = load_generation_data(date_filter="2022-07-18")
S, R, P_RT, K, K0, M1, M2 = load_parameters(I, T, generation_data)
P_DA, P_PN = load_price_data()

✅ 총 4개 파일을 불러왔습니다: 1201.csv, 137.csv, 397.csv, 514.csv
📊 데이터 Shape: I=4, T=24, S=20
✅ 시뮬레이션 초기화 완료: S=20, Randomness='high', M1=486.00, M2=924.00


In [26]:
def optimize_without(target_i, R, K, K0, P_DA, P_RT, P_PN, I, T, S):
    I_set = [i for i in range(I) if i != target_i]
    M1 = np.maximum(R[I_set], K[I_set, None, None]).max()
    M2 = max(R[I_set].sum(axis=0).max(), K[I_set].sum())

    model = gp.Model(f"set_without_{target_i}")
    model.setParam("MIPGap", 1e-7)
    model.setParam("OutputFlag", 0)

    x = model.addVars(I_set, T, vtype=GRB.CONTINUOUS, lb=0, name="x")
    ep = model.addVars(I_set, T, S, vtype=GRB.CONTINUOUS, name="e_plus")
    em = model.addVars(I_set, T, S, vtype=GRB.CONTINUOUS, name="e_minus")
    yp = model.addVars(I_set, T, S, vtype=GRB.CONTINUOUS, lb=0, name="y_plus")
    ym = model.addVars(I_set, T, S, vtype=GRB.CONTINUOUS, lb=0, name="y_minus")
    z = model.addVars(I_set, T + 1, S, vtype=GRB.CONTINUOUS, name="z")
    zc = model.addVars(I_set, T, S, vtype=GRB.CONTINUOUS, name="z_charge")
    zd = model.addVars(I_set, T, S, vtype=GRB.CONTINUOUS, name="z_discharge")
    d = model.addVars(I_set, I_set, T, S, vtype=GRB.CONTINUOUS, lb=0, name="d")

    p1 = model.addVars(I_set, T, S, vtype=GRB.BINARY, name="p1")
    p2 = model.addVars(I_set, T, S, vtype=GRB.BINARY, name="p2")
    p3 = model.addVars(I_set, T, S, vtype=GRB.BINARY, name="p3")
    p4 = model.addVars(I_set, T, S, vtype=GRB.BINARY, name="p4")

    obj = gp.quicksum(P_DA[t] * x[i, t] for i in I_set for t in range(T)) + gp.quicksum(
        (1 / S) * (
            P_RT[t, s] * gp.quicksum(ep[i, t, s] for i in I_set) -
            P_PN[t] * gp.quicksum(em[i, t, s] for i in I_set)
        )
        for t in range(T) for s in range(S)
    )
    model.setObjective(obj, GRB.MAXIMIZE)

    for i, t, s in product(I_set, range(T), range(S)):
        model.addConstr(R[i, t, s] - x[i, t] == yp[i, t, s] - ym[i, t, s] + zc[i, t, s] - zd[i, t, s])
        model.addConstr(yp[i, t, s] <= R[i, t, s])
        model.addConstr(zd[i, t, s] <= z[i, t, s])
        model.addConstr(zc[i, t, s] <= K[i] - z[i, t, s])
        model.addConstr(yp[i, t, s] <= M1 * p3[i, t, s])
        model.addConstr(ym[i, t, s] <= M1 * (1 - p3[i, t, s]))
        model.addConstr(ym[i, t, s] <= M1 * p2[i, t, s])
        model.addConstr(zc[i, t, s] <= M1 * (1 - p2[i, t, s]))
        model.addConstr(zc[i, t, s] <= M1 * p1[i, t, s])
        model.addConstr(zd[i, t, s] <= M1 * (1 - p1[i, t, s]))
        model.addConstr(z[i, t, s] <= K[i])
        model.addConstr(z[i, t + 1, s] == z[i, t, s] + zc[i, t, s] - zd[i, t, s])

    for i, s in product(I_set, range(S)):
        model.addConstr(z[i, 0, s] == K0[i])

    for i, t, s in product(I_set, range(T), range(S)):
        model.addConstr(ep[i, t, s] == yp[i, t, s] - gp.quicksum(d[i, j, t, s] for j in I_set if j != i))
        model.addConstr(em[i, t, s] == ym[i, t, s] - gp.quicksum(d[j, i, t, s] for j in I_set if j != i))
        model.addConstr(gp.quicksum(ep[i, t, s] for i in I_set) <= M2 * p4[i, t, s])
        model.addConstr(gp.quicksum(em[i, t, s] for i in I_set) <= M2 * (1 - p4[i, t, s]))
        model.addConstr(d[i, i, t, s] == 0)

    model.optimize()

    i_map = {i: idx for idx, i in enumerate(I_set)}

    yp_vals = np.array([[[yp[i, t, s].X for s in range(S)] for t in range(T)] for i in I_set])
    ym_vals = np.array([[[ym[i, t, s].X for s in range(S)] for t in range(T)] for i in I_set])
    d_vals = np.array([[[[d[i, j, t, s].X for s in range(S)] for t in range(T)] for j in I_set] for i in I_set])

    return yp_vals, ym_vals, d_vals, i_map

In [27]:
def optimize_without_forall(R, K, K0, P_DA, P_RT, P_PN, I, T, S):
    yp_without = {}
    ym_without = {}
    d_without = {}
    i_map_without = {}

    for target_i in tqdm(range(I), desc="Solving settlement model for each target DER"):
        yp_vals, ym_vals, d_vals, i_map = optimize_without(
            target_i, R, K, K0, P_DA, P_RT, P_PN, I, T, S
        )
        yp_without[target_i] = yp_vals
        ym_without[target_i] = ym_vals
        d_without[target_i] = d_vals
        i_map_without[target_i] = i_map

    return yp_without, ym_without, d_without, i_map_without

In [28]:
yp_without, ym_without, d_without, i_map_without = optimize_without_forall(R, K, K0, P_DA, P_RT, P_PN, I, T, S)

Solving settlement model for each target DER:   0%|          | 0/4 [00:00<?, ?it/s]

Set parameter MIPGap to value 1e-07


Solving settlement model for each target DER:  25%|██▌       | 1/4 [00:03<00:10,  3.40s/it]

Set parameter MIPGap to value 1e-07


Solving settlement model for each target DER:  50%|█████     | 2/4 [00:05<00:05,  2.58s/it]

Set parameter MIPGap to value 1e-07


Solving settlement model for each target DER:  75%|███████▌  | 3/4 [00:07<00:02,  2.43s/it]

Set parameter MIPGap to value 1e-07


Solving settlement model for each target DER: 100%|██████████| 4/4 [00:09<00:00,  2.43s/it]


In [41]:
def compute_rdc_coefficients_and_rho_from_all(
    yp_without, ym_without, d_without, i_map_without,
    P_RT, P_PN, T, S, I
):
    # 각 DER별 시간·시나리오별 RDC 함수 계수 [intercept, slope]
    rdc_coefficients_all = np.full((I, T, S, 2), np.nan)
    rho_plus_func_all = np.full((I, T, S, 2), np.nan)   # [a_plus, b_plus]
    rho_minus_func_all = np.full((I, T, S, 2), np.nan)  # [a_minus, b_minus]

    for target_i in range(I):
        yp_vals = yp_without[target_i]
        ym_vals = ym_without[target_i]
        d_vals = d_without[target_i]
        i_map = i_map_without[target_i]

        for t in range(T):
            for s in range(S):
                total_supply = sum(yp_vals[i_map[i], t, s] for i in i_map)
                total_demand = sum(ym_vals[i_map[i], t, s] for i in i_map)

                given_profit = received_profit = realized_supply = realized_demand = 0
                for i in i_map:
                    for j in i_map:
                        if i == j: continue
                        given_profit += d_vals[i_map[i], i_map[j], t, s] * P_PN[t]
                        received_profit += d_vals[i_map[j], i_map[i], t, s] * P_RT[t, s]
                        realized_supply += d_vals[i_map[i], i_map[j], t, s]
                        realized_demand += d_vals[i_map[j], i_map[i], t, s]

                # Case 1: RDC 불가능 → fallback 함수
                BIG_M_POS = 1e6
                BIG_M_NEG = -1e6

                if realized_demand <= 1e-4 or realized_supply <= 1e-4:
                    # RDC 불가능 → 매우 큰 기울기 (매우 비싼 가격으로 유도)
                    rho_plus_func_all[target_i, t, s, :] = [BIG_M_NEG, 0.0]
                    rho_minus_func_all[target_i, t, s, :] = [BIG_M_POS, 0.0]
                    rdc_coefficients_all[target_i, t, s, :] = [np.nan, np.nan]
                    continue

                # Case 2: RDC 추정 가능
                a_d = P_PN[t]
                b_d = 2 * (a_d * realized_demand - received_profit) / (realized_demand ** 2)
                a_s = P_RT[t, s]
                b_s = 2 * (given_profit - a_s * realized_supply) / (realized_supply ** 2)

                x_max = 1.1 * max(total_demand, total_supply)
                q0_list = np.linspace(-5, x_max, 100)
                clearing_prices = []

                for q0 in q0_list:
                    denom = b_d + b_s
                    if abs(denom) < 1e-6:
                        clearing_prices.append(np.nan)
                        continue
                    q_cleared = (a_d - a_s + b_s * q0) / denom
                    p_cleared = a_d - b_d * q_cleared
                    clearing_prices.append(p_cleared)

                mask = ~np.isnan(clearing_prices)
                q0_array = np.array(q0_list)[mask].reshape(-1, 1)
                p_array = np.array(clearing_prices)[mask]

                if len(q0_array) < 2:
                    rho_plus_func_all[target_i, t, s, :] = [BIG_M_NEG, 0.0]
                    rho_minus_func_all[target_i, t, s, :] = [BIG_M_POS, 0.0]
                    rdc_coefficients_all[target_i, t, s, :] = [np.nan, np.nan]
                    continue

                # 다항 회귀로 선형 RDC 함수 도출
                X_poly = PolynomialFeatures(degree=1).fit_transform(q0_array)
                model = LinearRegression().fit(X_poly, p_array)
                intercept, slope = model.intercept_, model.coef_[1]

                # 동일한 RDC 함수 적용
                rho_plus_func_all[target_i, t, s, :] = [intercept, slope]
                rho_minus_func_all[target_i, t, s, :] = [intercept, slope]
                rdc_coefficients_all[target_i, t, s, :] = [intercept, slope]

    return rdc_coefficients_all, rho_plus_func_all, rho_minus_func_all

In [42]:
rdc, rho_plus, rho_minus = compute_rdc_coefficients_and_rho_from_all(
    yp_without, ym_without, d_without, i_map_without,
    P_RT, P_PN, T, S, I
)

In [49]:
# 인덱스 설정
target_i = 2
t = 14
s = 2

# 가격 함수 계수 추출
ap, bp = rho_plus[target_i, t, s, :]
am, bm = rho_minus[target_i, t, s, :]

print(f"ρ⁺(d) = {ap:.2f} + {bp:.2f}·d")
print(f"ρ⁻(d) = {am:.2f} + {bm:.2f}·d")

ρ⁺(d) = 173.21 + -0.94·d
ρ⁻(d) = 173.21 + -0.94·d


Model (Non linear)

In [ ]:
# model = gp.Model("piecewise")
# model.setParam("MIPGap", 1e-7)

# x = model.addVars(T, vtype=GRB.CONTINUOUS, lb=0, name="x")

# yp = model.addVars(T, S, vtype=GRB.CONTINUOUS, lb=0, name="y_plus")
# ym = model.addVars(T, S, vtype=GRB.CONTINUOUS, lb=0, name="y_minus")
# z = model.addVars(T + 1, S, vtype=GRB.CONTINUOUS, lb=0, name="z")
# zc = model.addVars(T, S, vtype=GRB.CONTINUOUS, lb=0, name="z_charge")
# zd = model.addVars(T, S, vtype=GRB.CONTINUOUS, lb=0, name="z_discharge")
# dp = model.addVars(T, S, vtype=GRB.CONTINUOUS, lb=0, name="d_plus")
# dm = model.addVars(T, S, vtype=GRB.CONTINUOUS, lb=0, name="d_minus")

# p1 = model.addVars(T, S, vtype=GRB.BINARY, name="p1")
# p2 = model.addVars(T, S, vtype=GRB.BINARY, name="p2")
# p3 = model.addVars(T, S, vtype=GRB.BINARY, name="p3")
# p4 = model.addVars(T, S, vtype=GRB.BINARY, name="p4")

# model.update()

# ap = {}
# bp = {}
# am = {}
# bm = {}

# for t in range(T):
#     for s in range(S):
#         ap[t, s]  = rho_plus[target_i, t, s, 0]
#         bp[t, s]  = rho_plus[target_i, t, s, 1]
#         am[t, s] = rho_minus[target_i, t, s, 0]
#         bm[t, s] = rho_minus[target_i, t, s, 1]

# obj = gp.quicksum(P_DA[t] * x[t] for t in range(T)) + \
#       gp.quicksum(
#           (1/S) * (
#               P_RT[t, s] * yp[t, s] - P_PN[t] * ym[t, s] +
#               ap[t, s] * dp[t, s] + bp[t, s] * dp[t, s] * dp[t, s] -
#               am[t, s] * dm[t, s] - bm[t, s] * dm[t, s] * dm[t, s]
#           )
#           for t in range(T) for s in range(S)
#       )

# model.setObjective(obj, GRB.MAXIMIZE)

# for t, s in product(range(T), range(S)):
#     model.addConstr(R[target_i, t, s] - x[t] == yp[t, s] - ym[t, s] + zc[t, s] - zd[t, s] + dp[t, s] - dm[t, s])  # (8b)
#     model.addConstr(R[target_i, t, s] >= yp[t, s] + dp[t, s])  # (8c)
#     model.addConstr(ym[t, s] >= dm[t, s])                      # (8d)
#     model.addConstr(z[t + 1, s] == z[t, s] + zc[t, s] - zd[t, s])  # (8e)

#     # (8f)
#     model.addConstr(zd[t, s] <= z[t, s])
#     model.addConstr(zc[t, s] <= K[target_i] - z[t, s])
#     model.addConstr(z[t, s] >= 0)
#     model.addConstr(z[t, s] <= K[target_i])

#     # (8g)
#     model.addConstr(yp[t, s] <= M1 * p1[t, s])
#     model.addConstr(ym[t, s] <= M1 * (1 - p1[t, s]))

#     # (8h)
#     model.addConstr(ym[t, s] <= M1 * p2[t, s])
#     model.addConstr(zc[t, s] <= M1 * (1 - p2[t, s]))

#     # (8i)
#     model.addConstr(zc[t, s] <= M1 * p3[t, s])
#     model.addConstr(zd[t, s] <= M1 * (1 - p3[t, s]))

#     # (8j)
#     model.addConstr(dp[t, s] <= M1 * p4[t, s])
#     model.addConstr(dm[t, s] <= M1 * (1 - p4[t, s]))

# for s in range(S):
#     model.addConstr(z[0, s] == K0[target_i])

# model.optimize()

# if model.status == GRB.OPTIMAL:
#     print(f"Optimal solution found! Objective value: {model.objVal}")
# else:
#     print("No optimal solution found.")
    
# x_vals = np.array([x[t].X for t in range(T)])
# yp_vals = np.array([[yp[t, s].X for s in range(S)] for t in range(T)])
# ym_vals = np.array([[ym[t, s].X for s in range(S)] for t in range(T)])
# z_vals  = np.array([[z[t, s].X for s in range(S)] for t in range(T+1)])
# zc_vals = np.array([[zc[t, s].X for s in range(S)] for t in range(T)])
# zd_vals = np.array([[zd[t, s].X for s in range(S)] for t in range(T)])
# dp_vals = np.array([[dp[t, s].X for s in range(S)] for t in range(T)])
# dm_vals = np.array([[dm[t, s].X for s in range(S)] for t in range(T)])

Model (Linear)

In [71]:
target_i = 3
# === 모델 정의 ===
model = gp.Model("PWL_Internal_Price")
model.setParam("MIPGap", 1e-7)

# === 변수 정의 ===
x   = model.addVars(T, vtype=GRB.CONTINUOUS, lb=0, name="x")
yp  = model.addVars(T, S, vtype=GRB.CONTINUOUS, lb=0, name="y_plus")
ym  = model.addVars(T, S, vtype=GRB.CONTINUOUS, lb=0, name="y_minus")
z   = model.addVars(T+1, S, vtype=GRB.CONTINUOUS, lb=0, name="z")
zc  = model.addVars(T, S, vtype=GRB.CONTINUOUS, name="z_charge")
zd  = model.addVars(T, S, vtype=GRB.CONTINUOUS, name="z_discharge")
dp  = model.addVars(T, S, vtype=GRB.CONTINUOUS, name="d_plus")
dm  = model.addVars(T, S, vtype=GRB.CONTINUOUS, name="d_minus")
p1  = model.addVars(T, S, vtype=GRB.BINARY, name="p1")
p2  = model.addVars(T, S, vtype=GRB.BINARY, name="p2")
p3  = model.addVars(T, S, vtype=GRB.BINARY, name="p3")
p4  = model.addVars(T, S, vtype=GRB.BINARY, name="p4")

model.update()

ap = {}
bp = {}
am = {}
bm = {}

for t in range(T):
    for s in range(S):
        ap[t, s]  = rho_plus[target_i, t, s, 0]
        bp[t, s]  = rho_plus[target_i, t, s, 1]
        am[t, s] = rho_minus[target_i, t, s, 0]
        bm[t, s] = rho_minus[target_i, t, s, 1]

# === 목적함수: 선형 부분 + pwl ===
obj = gp.quicksum(P_DA[t] * x[t] for t in range(T)) + \
      gp.quicksum((1/S) * (P_RT[t, s] * yp[t, s] - P_PN[t] * ym[t, s]) for t, s in product(range(T), range(S)))
model.setObjective(obj, GRB.MAXIMIZE)

# === 제약식 ===
for t, s in product(range(T), range(S)):
    model.addConstr(R[target_i, t, s] - x[t] == yp[t, s] - ym[t, s] + zc[t, s] - zd[t, s] + dp[t, s] - dm[t, s])
    model.addConstr(yp[t, s] + dp[t, s] <= R[target_i, t, s])
    model.addConstr(ym[t, s] >= dm[t, s])
    model.addConstr(z[t+1, s] == z[t, s] + zc[t, s] - zd[t, s])
    model.addConstr(zd[t, s] <= z[t, s])
    model.addConstr(zc[t, s] <= K[target_i] - z[t, s])
    model.addConstr(z[t, s] >= 0)
    model.addConstr(z[t, s] <= K[target_i])
    model.addConstr(yp[t, s] <= M1 * p1[t, s])
    model.addConstr(ym[t, s] <= M1 * (1 - p1[t, s]))
    model.addConstr(ym[t, s] <= M1 * p2[t, s])
    model.addConstr(zc[t, s] <= M1 * (1 - p2[t, s]))
    model.addConstr(zc[t, s] <= M1 * p3[t, s])
    model.addConstr(zd[t, s] <= M1 * (1 - p3[t, s]))
    model.addConstr(dp[t, s] <= M1 * p4[t, s])
    model.addConstr(dm[t, s] <= M1 * (1 - p4[t, s]))

for s in range(S):
    model.addConstr(z[0, s] == K0[target_i])

# === Piecewise Linear Objective 적용 ===
for t, s in product(range(T), range(S)):
    a_p, b_p = rho_plus[target_i, t, s]
    a_m, b_m = rho_minus[target_i, t, s]

    MAX = max(R[target_i, t, s], K[target_i])

    model.setPWLObj(
        dp[t, s],
        [0, MAX],
        [a_p * 0, a_p * MAX + b_p * MAX]
    )

    model.setPWLObj(
        dm[t, s],
        [0, MAX],
        [-a_m * 0, -a_m * MAX - b_m * MAX]
    )

# === 최적화 ===
model.optimize()

yp_vals = np.array([[yp[t, s].X for s in range(S)] for t in range(T)])
ym_vals = np.array([[ym[t, s].X for s in range(S)] for t in range(T)])
dp_vals = np.array([[dp[t, s].X for s in range(S)] for t in range(T)])
dm_vals = np.array([[dm[t, s].X for s in range(S)] for t in range(T)])

Set parameter MIPGap to value 1e-07
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.5.0 24F74)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Non-default parameters:
MIPGap  1e-07

Optimize a model with 7700 rows, 5324 columns and 17780 nonzeros
Model fingerprint: 0x323ae5ac
Model has 960 piecewise-linear objective terms
Variable types: 3404 continuous, 1920 integer (1920 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+02]
  Objective range  [2e+00, 3e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+02]
Presolve removed 2160 rows and 956 columns
Presolve time: 0.01s
Presolved: 5540 rows, 4368 columns, 13992 nonzeros
Variable types: 2876 continuous, 1492 integer (1492 binary)
Found heuristic solution: objective 535728.04785

Root relaxation: objective 5.420007e+05, 1132 iterations, 0.01 seconds (0.02 work units)

    Nodes    |    Current Node    |     Objective Bounds      |   

In [73]:
for t, s in product(range(T), range(S)):
    if(dp_vals[t,s] > 0): 
        print(dp_vals[t, s])

1.0
2.0
2.0
1.0
2.0
1.0
2.0
2.0
1.0
1.0
1.0
2.0
1.0
1.0
1.0
2.0
5.0
5.0
4.0
4.0
3.0
5.0
1.0
5.0
2.0
3.0
4.0
2.0
1.0
3.0
5.0
4.0
1.0
5.0
3.0
5.0
17.0
5.0
8.0
24.0
20.0
25.0
11.0
16.0
19.0
18.0
9.0
13.0
9.0
24.0
27.0
5.0
20.0
11.0
18.0
9.0
42.0
27.0
23.0
56.0
57.0
40.0
18.0
57.0
47.0
47.0
18.0
51.0
20.0
41.0
45.0
46.0
28.0
47.0
55.0
38.0
14.0
16.0
15.0
19.0
16.0
57.0
10.0
23.0
63.0
69.0
46.0
70.0
11.0
68.0
28.0
58.0
59.0
62.0
37.0
67.0
32.0
87.0
61.0
92.0
13.0
70.0
57.0
51.0
32.0
37.0
31.0
28.0
76.0
52.0
17.0
74.0
54.0
71.0
53.0
43.0
150.0
21.0
143.0
143.0
70.0
46.0
101.0
41.0
125.0
51.0
56.0
141.0
139.0
89.0
134.0
65.0
152.0
76.0
57.0
23.0
132.0
70.0
101.0
90.0
100.0
21.0
123.0
124.0
80.0
128.0
48.0
86.0
62.0
89.0
50.0
111.0
23.0
20.0
21.0
33.0
28.0
58.0
13.0
41.0
35.0
26.0
17.0
19.0
9.0
45.0
42.0
30.0
45.0
13.0
57.0
52.0
42.0
27.0
29.0
33.0
74.0
69.0
42.0
47.0
69.0
75.0
53.0
61.0
33.0
72.0
67.0
47.0
69.0
68.0
65.0
18.0
46.0
71.0
62.0
38.0
73.0
25.0
30.0
71.0
61.0
67.0
9.0
33.0
24.0
28.

In [1]:
for t, s in product(range(T), range(S)):
    if(dm_vals[t,s]> 0): 
        print(dm_vals[t, s])

NameError: name 'product' is not defined